In [1]:
# Define numeric season and adjust year
def get_season_info(row):
    month = row['month']
    year = row['year']
    
    if month in [12, 1, 2]:
        num_season = 4  # Summer
        if month == 12:
            year_season = int(f"{year}4")
        else:  # Jan, Feb → belongs to previous Dec
            year_season = int(f"{year-1}4")
    elif month in [3, 4, 5]:
        num_season = 1  # Autumn
        year_season = int(f"{year}1")
    elif month in [6, 7, 8]:
        num_season = 2  # Winter
        year_season = int(f"{year}2")
    else:  # [9, 10, 11]
        num_season = 3  # Spring
        year_season = int(f"{year}3")
        
    return pd.Series({'num_season': num_season, 'year_season': year_season})

# Define seasons for Southern Hemisphere
def month_to_season(month):
    if month in [9, 10, 11]:
        return 'Spring'
    elif month in [12, 1, 2]:
        return 'Summer'
    elif month in [3, 4, 5]:
        return 'Autumn'
    else:
        return 'Winter'

In [2]:
def generate_profiles(df):
    """
    Generate various temperature profiles from the input DataFrame.

    Parameters:
    - df: DataFrame

    Returns:
    - DataFrames:
      year_profile  : DataFrame (altitude_bin-year avg),
      year_season_profile : DataFrame (grouped by year_season),
      season_profile: DataFrame (grouped by season),
      mean_profile: DataFrame (altitude_bin mean values)
    """

    # Group by altitude_bin and year_season
    year_profile = df.groupby(['altitude_bin', 'year'])['ktemp'].mean().reset_index()

    # Group by altitude_bin and year_season
    year_season_profile = df.groupby(['altitude_bin', 'year_season'])['ktemp'].mean().reset_index()

    # Group by season
    season_profile = df.groupby(['altitude_bin', 'season'])['ktemp'].mean().reset_index()

    # Mean profile across all time
    mean_profile = df.groupby('altitude_bin')[['ktemp']].mean().reset_index()

    return year_profile, year_season_profile,season_profile, mean_profile

In [3]:
def sen_slope(df):
    """
    Calculate Sen's slope and intercept for a given time series.

    Parameters
    ----------
    df : pd.DataFrame
        Must contain columns ['year_month', 'ktemp'].
        - 'year_month' can be datetime or period type.
        - 'ktemp' is the temperature variable.

    Returns
    -------
    pd.DataFrame
        With columns ['sen_slope', 'intercept'].
    """
    # Convert year_month to fractional year (YYYY + fraction of month)
    x = df['year_month'].dt.to_timestamp().map(lambda d: d.year + (d.month - 1) / 12.0).values
    y = df['ktemp'].values

    slope, intercept, _, _ = theilslopes(y, x)

    return pd.DataFrame([{'sen_slope': slope, 'intercept': intercept}])

In [4]:
def sen_slope_by_altitude(df):
    """
    Computes Sen's slope of temperature vs. time for each altitude_bin.

    Parameters:
    - df: DataFrame with columns ['altitude_bin', 'year_month', 'ktemp']
              'year_month' must be a pandas datetime (e.g., Period or Timestamp)
    - compute_sen_slope: Function that computes Sen's slope given (x, y)

    Returns:
    - slopes_df: DataFrame with columns ['altitude_bin', 'sen_slope']
    """
    slopes = []
    altitudes = sorted(df['altitude_bin'].unique())

    for alt in altitudes:
        df_alt = df[df['altitude_bin'] == alt].dropna(subset=['ktemp'])
        
        # Convert 'year_month' to fractional year: 2001-03 -> 2001.1667
        x = df_alt['year_month'].dt.to_timestamp().map(lambda d: d.year + (d.month - 1) / 12.0).values
        y = df_alt['ktemp'].values

        if len(x) > 1:  # Ensure enough points to compute slope
            # from scipy.stats import theilslopes
            slope, intercept, _, _ = theilslopes(y, x)
        else:
            slope = np.nan  # Not enough data to compute slope

        slopes.append({'altitude_bin': alt, 'sen_slope': slope, 'intercept': intercept})

    return pd.DataFrame(slopes)

In [5]:
def run_mk_tests(df, mk):
    """
    Runs the Mann-Kendall test for each altitude/time series and returns results as a DataFrame.

    Parameters:
    - df: DataFrame with columns ['year_month', 'ktemp']
    - mk: A module or object with the method original_test(series)

    Returns:
    - DataFrame with columns:
        ['trend', 'h', 'p', 'z', 'tau', 's', 'var_s', 'slope', 'intercept', 'n']
    """
    results = []

    # Group by 'year_month' to get average per month
    ts = df.sort_values('year_month').groupby('year_month')['ktemp'].mean().sort_index()

    # Convert PeriodIndex to datetime if needed
    if hasattr(ts.index, 'to_timestamp'):
        ts.index = ts.index.to_timestamp()

    # Drop NA values
    ts = ts.dropna()

    # Apply Mann-Kendall Test + Sen's Slope if enough data points
    if len(ts) > 10:  # Require minimum data points
        mk_result = mk.original_test(ts.values)
        results.append({
            'trend': mk_result.trend,
            'h': getattr(mk_result, 'h', None),  # some implementations return h
            'p': mk_result.p,
            'z': mk_result.z,
            'tau': mk_result.Tau,
            's': mk_result.s,
            'var_s': mk_result.var_s,
            'slope': mk_result.slope,
            'intercept': mk_result.intercept,
            'n': len(ts)
        })

    return pd.DataFrame(results)

In [6]:
def run_mk_tests_by_altitude(df, mk):
    """
    Runs the Mann-Kendall test for each specified altitude and returns the results as a DataFrame.

    Parameters:
    - agg_df: DataFrame with columns ['altitude_bin', 'year_month', 'ktemp']
    - altitudes_to_plot: List of altitudes to test (e.g., [300, 350, 400])
    - mk: A module or object with the method original_test(series)

    Returns:
    - DataFrame with columns:
        ['altitude_bin', 'trend', 'h', 'p', 'z', 'Tau', 's', 'var_s', 'slope', 'intercept', 'n']
    """
    results = []

    for alt in sorted(df['altitude_bin'].unique()):
        df_alt = df[df['altitude_bin'] == alt].sort_values('year_month')
    
        # Group by 'year_month' to get average per month
        ts = df_alt.groupby('year_month')['ktemp'].mean().sort_index()
        
        # Convert PeriodIndex to datetime for compatibility
        ts.index = ts.index.to_timestamp()
        
        # Drop NA
        ts = ts.dropna()
    
        # Apply Mann-Kendall Test + Sen's Slope
        if len(ts) > 10:  # Require minimum data points
            mk_result = mk.original_test(ts.values)
            results.append({
                'altitude_bin': alt,
                'slope': mk_result.slope,
                'trend': mk_result.trend,
                'p': mk_result.p,
                'z': mk_result.z,
                'tau': mk_result.Tau,
                's': mk_result.s,
                'var_s': mk_result.var_s,
                'slope': mk_result.slope,
                'intercept': mk_result.intercept,
                'n': len(ts)
            })

    return pd.DataFrame(results)

In [7]:
def sequential_mk_test(X, alpha=0.05):
    """
    Performs the Sequential Mann–Kendall (S-MK) test.
    
    Parameters:
    - x: 1D array or list of values (time series)
    - alpha: significance level (e.g., 0.05)

    Returns:
    - dict containing UF, UB, crossing points, and critical value
    """
    x = np.array(X)
    n = len(x)
    UF = np.zeros(n)
    UB = np.zeros(n)

    # Forward series (UF)
    S = 0
    var_s = np.zeros(n)
    for k in range(1, n):
        s = 0
        for j in range(k):
            if x[k] > x[j]:
                s += 1
            elif x[k] < x[j]:
                s -= 1
        S += s
        var_s[k] = (k * (k + 1) * (2 * k + 5)) / 18
        UF[k] = S / np.sqrt(var_s[k]) if var_s[k] > 0 else 0

    # Backward series (UB)
    x_reversed = x[::-1]
    S = 0
    for k in range(1, n):
        s = 0
        for j in range(k):
            if x_reversed[k] > x_reversed[j]:
                s += 1
            elif x_reversed[k] < x_reversed[j]:
                s -= 1
        S += s
        UB[k] = -S / np.sqrt(var_s[k]) if var_s[k] > 0 else 0

    UB = UB[::-1]  # Flip UB to match forward time

    # Critical Z value
    z_crit = norm.ppf(1 - alpha / 2)

    # Detect crossing points
    cross_indices = np.where((UF - UB) > z_crit)[0]

    return {
        'UF': UF,
        'UB': UB,
        'z_crit': z_crit,
        'cross_points': cross_indices}

In [8]:
def run_sequential_mk_by_altitude(df, alpha=0.05):
    results = []

    for alt in sorted(df['altitude_bin'].unique()):
        df_alt = df[df['altitude_bin'] == alt]
        ts = df_alt.groupby('year_month')['ktemp'].mean().sort_index()

        if hasattr(ts.index, 'to_timestamp'):
            ts.index = ts.index.to_timestamp()

        ts = ts.dropna()

        if len(ts) > 10:
            smk_result = sequential_mk_test(ts.values, alpha=alpha)
            results.append({
                'altitude_bin': alt,
                'cross_points': smk_result['cross_points'],
                'UF': smk_result['UF'],
                'UB': smk_result['UB'],
                'z_crit': smk_result['z_crit'],
                'n': len(ts)
            })

    return results

In [9]:
def ITA_alt(df, split_ratio=0.5):
    ita_results = []

    for alt in sorted(df['altitude_bin'].unique()):
        df_alt = df[df['altitude_bin'] == alt]
        ts = df_alt.groupby('year_month')['ktemp'].mean().sort_index()

        # Convert period to timestamp if needed
        if hasattr(ts.index, 'to_timestamp'):
            ts.index = ts.index.to_timestamp()

        ts_values = ts.dropna().values
        ts_values = ts_values[~np.isnan(ts_values)]  # ensure no NaNs
        N = len(ts_values)

        if N < 10:
            continue  # skip altitudes with insufficient data

        data_sorted = np.sort(ts_values)
        split = int(N * split_ratio)
        X = data_sorted[:split]
        Y = data_sorted[-split:]

        fit = linregress(X, Y)
        ita_slope = fit.slope

        ita_results.append({
            'altitude_bin': alt,
            'ita_slope': ita_slope,
            'n': N
        })

    ita_df = pd.DataFrame(ita_results)

    return ita_df


In [10]:
def ITA(df, split_ratio=0.5):
    """
    Perform Innovative Trend Analysis (ITA) on a time series.

    Parameters
    ----------
    series : pd.Series
        Time series data with a datetime or period index.
    split_ratio : float, default=0.5
        Fraction of data to split into lower and upper groups (usually 0.5).

    Returns
    -------
    pd.DataFrame
        With columns:
        - 'ita_slope': slope of ITA regression line
        - 'n': number of data points used
    """
    ts = df.sort_index()

    if hasattr(ts.index, 'to_timestamp'):
        ts.index = ts.index.to_timestamp()

    ts = ts.dropna()
    ts_values = pd.to_numeric(ts['ktemp'], errors='coerce').values
    ts_values = ts_values[~np.isnan(ts_values)]
    N = len(ts_values)

    if N < 10:
        print("Not enough data points.")
        return pd.DataFrame()

    data_sorted = np.sort(ts_values)
    split = int(N * split_ratio)
    X = data_sorted[:split]
    Y = data_sorted[-split:]

    fit = linregress(X, Y)
    ita_slope = fit.slope

    result = pd.DataFrame([{
        'ita_slope': ita_slope,
        'n': N
    }])

    return result

In [11]:
def analyze_trends_count(smk_results):
    rows = []

    for res in smk_results:
        alt = res['altitude_bin']
        UF = res['UF']
        z_crit = res['z_crit']
        n = res['n']
        cross_points = res['cross_points']

        # Check points exceeding significance threshold
        significant_up = np.where(UF > z_crit)[0]
        significant_down = np.where(UF < -z_crit)[0]

        rows.append({
            'altitude_bin': alt,
            'increasing months': len(significant_up),
            'decreasing months': len(significant_down),
            'n_change_points': len(cross_points),
            'n': n
        })

    return pd.DataFrame(rows)

In [12]:
def run_sequential_mk(df, alpha=0.05):
    """
    Run the Sequential Mann-Kendall test on a temperature time series.

    Parameters:
    - df: DataFrame with 'year_month' as period[M] and 'ktemp' as float
    - alpha: significance level (default 0.05)

    Returns:
    - DataFrame with UF, UB, and cross_point flags
    - Dictionary with additional info: z_crit, cross_points, n
    """

    df = df.sort_values('year_month').reset_index(drop=True)
    ts = df['ktemp'].values
    n = len(ts)

    UF = [0] * n
    var_s = [0] * n

    for i in range(1, n):
        S = 0
        for j in range(i):
            if ts[i] > ts[j]:
                S += 1
            elif ts[i] < ts[j]:
                S -= 1
        var_s[i] = i * (i + 1) * (2 * i + 5) / 18
        UF[i] = S / np.sqrt(var_s[i]) if var_s[i] > 0 else 0

    UB = [0] * n
    ts_rev = ts[::-1]
    for i in range(1, n):
        S = 0
        for j in range(i):
            if ts_rev[i] > ts_rev[j]:
                S += 1
            elif ts_rev[i] < ts_rev[j]:
                S -= 1
        UB[i] = -S / np.sqrt(var_s[i]) if var_s[i] > 0 else 0
    UB = UB[::-1]

    z_crit = norm.ppf(1 - alpha / 2)

    cross_points = [i for i in range(n) if abs(UF[i] - UB[i]) < 1e-6]
    cross_point_flags = [i in cross_points for i in range(n)]

    result_df = pd.DataFrame({
        'year_month': df['year_month'].astype(str),
        'UF': UF,
        'UB': UB,
        'cross_point': cross_point_flags
    })

    return result_df

In [13]:
# def seasonal_physics_loss(y_pred, short_period=10, long_period=132):
#     """
#     Penalizes deviation from known sine-based seasonal structure.

#     Parameters:
#     y_pred (np.ndarray): Predicted values
#     t (np.ndarray): Time axis (same shape as y_pred)
#     short_period (int): Short seasonality period (e.g., 10 months)
#     long_period (int): Long seasonality period (e.g., 132 months)

#     Returns:
#     float: Mean squared residual from expected harmonic structure
#     """
#     t = np.arange(len(y_pred))
    
#     short_season = np.sin(2 * np.pi * t / short_period)
#     long_season = np.sin(2 * np.pi * (t + 20) / long_period)
    
#     # Simple least-squares fit
#     X = np.vstack([short_season, long_season, t, np.ones_like(t)]).T
#     coeffs, _, _, _ = np.linalg.lstsq(X, y_pred, rcond=None)
#     y_fit = X @ coeffs

#     # Residual between predicted and harmonic model
#     residual = y_pred - y_fit
#     return np.mean(residual ** 2)

In [14]:
def mse_loss(y_true, y_pred):
    """
    Calculate the Mean Squared Error loss.
    
    Parameters:
    y_true (np.ndarray): Ground truth values
    y_pred (np.ndarray): Predicted values
    
    Returns:
    float: Mean Squared Error
    """
    return np.mean((y_true - y_pred) ** 2)

In [15]:
# def ODE_loss(y_pred, a=0.01):
#     """
#     Physics-informed loss for: y'' + y = -a * t

#     Parameters:
#     y_pred (np.ndarray): Predicted values
#     t (np.ndarray): Time steps
#     a (float): Coefficient for the forcing term

#     Returns:
#     float: Physics residual loss
#     """

#     t = np.arrange(len(y_pred))
    
#     # Finite difference approximation for y''
#     dt = t[1] - t[0]
#     y_tt = (y_pred[:-2] - 2 * y_pred[1:-1] + y_pred[2:]) / (dt ** 2)

#     y_mid = y_pred[1:-1]
#     t_mid = t[1:-1]
#     # Compute residual of the differential equation: y'' + y + a*t = 0
#     residual = y_tt + y_mid + a * t_mid
#     # Mean squared residual
#     return np.mean(residual ** 2)

In [16]:
def ODE_loss_fn(a=0.01):
    def loss(y_true, y_pred):
        # Assume uniform time steps: t = 0, 1, 2, ..., N-1
        t = tf.range(tf.shape(y_pred)[1], dtype=tf.float32)
        dt = t[1] - t[0]

        # Slicing for finite difference (along sequence dimension)
        y_left = y_pred[:, :-2]
        y_mid = y_pred[:, 1:-1]
        y_right = y_pred[:, 2:]
        t_mid = t[1:-1]

        # Finite difference second derivative
        y_tt = (y_left - 2 * y_mid + y_right) / (dt ** 2)

        # Residual of the ODE: y'' + y + a*t = 0
        residual = y_tt + y_mid + a * t_mid  # broadcasting happens automatically
        return tf.reduce_mean(tf.square(residual))
    
    return loss

In [17]:
def seasonal_physics_loss_tf(short_period=10, long_period=132):
    def loss(y_true, y_pred):
        # Assume shape: (batch_size, time_steps)
        time_steps = tf.shape(y_pred)[1]
        t = tf.cast(tf.range(time_steps), tf.float32)  # shape: (time_steps,)

        # Build harmonic basis
        short_season = tf.sin(2 * np.pi * t / short_period)
        long_season = tf.sin(2 * np.pi * (t + 20) / long_period)

        # Design matrix: (time_steps, 4)
        X = tf.stack([short_season, long_season, t, tf.ones_like(t)], axis=1)  # shape: (T, 4)

        # Least squares fit for each sample in batch
        def single_residual(y_i):
            # y_i shape: (time_steps,)
            coeffs = tf.linalg.lstsq(X, tf.reshape(y_i, [-1, 1]), l2_regularizer=0.0, fast=False)  # (4,1)
            y_fit = tf.matmul(X, coeffs)  # (T,1)
            residual = tf.reshape(y_i, [-1, 1]) - y_fit
            return tf.reduce_mean(tf.square(residual))

        # Apply to each sample in the batch
        residuals = tf.map_fn(single_residual, y_pred, dtype=tf.float32)
        return tf.reduce_mean(residuals)  # Final scalar loss

    return loss


In [18]:
def combined_loss_fn(a=0.01, b=0, lambda_mse=1.0, lambda_ode=1.0):
    def loss(y_true, y_pred):
        # MSE loss
        mse = tf.reduce_mean(tf.square(y_true - y_pred))

        # ODE loss
        t = tf.range(tf.shape(y_pred)[1], dtype=tf.float32)
        dt = t[1] - t[0]

        y_left = y_pred[:, :-2]
        y_mid = y_pred[:, 1:-1]
        y_right = y_pred[:, 2:]
        t_mid = t[1:-1]

        y_tt = (y_left - 2 * y_mid + y_right) / (dt ** 2)
        residual = y_tt + y_mid + a * t_mid + b
        ode = tf.reduce_mean(tf.square(residual))

        # Combine losses
        return lambda_mse * mse + lambda_ode * ode
    return loss

In [19]:
# Point-wise metrics
def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def mse(y_true, y_pred):
    return np.mean((y_true - y_pred)**2)

def rmse(y_true, y_pred):
    return np.sqrt(mse(y_true, y_pred))

def mape(y_true, y_pred):
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    mask = denominator != 0
    return np.mean(np.abs(y_true - y_pred)[mask] / denominator[mask]) * 100

# MASE
def mase(y_true, y_pred, seasonality=1):
    naive_forecast = y_true[:-seasonality]
    scale = np.mean(np.abs(y_true[seasonality:] - naive_forecast))
    return np.mean(np.abs(y_true[seasonality:] - y_pred[seasonality:])) / scale

# NSE
def nse(y_true, y_pred):
    return 1 - np.sum((y_true - y_pred)**2) / np.sum((y_true - np.mean(y_true))**2)

# Forecast skill
def skill_score(y_true, y_pred, baseline_pred):
    model_error = mse(y_true, y_pred)
    baseline_error = mse(y_true, baseline_pred)
    return 1 - model_error / baseline_error

# Residual autocorrelation
def residual_acf(y_true, y_pred, nlags=20):
    residuals = y_true - y_pred
    return acf(residuals.flatten(), nlags=nlags)



def quantile_loss(y_true, y_pred, quantile=0.9):
    error = y_true - y_pred
    return np.mean(np.maximum(quantile * error, (quantile - 1) * error))


def picp(y_true, lower, upper):
    within_interval = np.logical_and(y_true >= lower, y_true <= upper)
    return np.mean(within_interval)




def dtw_distance(y_true, y_pred):
    return dtw.distance(y_true, y_pred)



def crps_score(y_true, y_samples):
    """
    y_true: shape (n_samples,)
    y_samples: shape (n_samples, n_ensemble_members)
    """
    return np.mean(crps_ensemble(y_true, y_samples))


def pinball_loss(y_true, y_pred, quantile):
    error = y_true - y_pred
    return np.mean(np.maximum(quantile * error, (quantile - 1) * error))




def residual_autocorrelation(y_true, y_pred, lags=20):
    residuals = y_true - y_pred
    return acf(residuals, nlags=lags)


def nash_sutcliffe_efficiency(y_true, y_pred):
    numerator = np.sum((y_true - y_pred)**2)
    denominator = np.sum((y_true - np.mean(y_true))**2)
    return 1 - (numerator / denominator)



In [20]:
def evaluate_forecast(y_true, y_pred, baseline_pred=None, title=None):
    y_true = y_true.reshape(-1)
    y_pred = y_pred.reshape(-1)

    print("Forecast Evaluation Metrics:")
    print(f"MAE     : {mae(y_true, y_pred):.4f}")
    print(f"MSE     : {mse(y_true, y_pred):.4f}")
    print(f"RMSE    : {rmse(y_true, y_pred):.4f}")
    print(f"MAPE    : {mape(y_true, y_pred):.2f}%")
    print(f"sMAPE   : {smape(y_true, y_pred):.2f}%")
    print(f"MASE    : {mase(y_true, y_pred):.4f}")
    print(f"NSE     : {nse(y_true, y_pred):.4f}")
    if baseline_pred is not None:
        print(f"Skill Score vs Baseline: {skill_score(y_true.reshape(y_true.shape[0], -1), y_pred.reshape(y_pred.shape[0], -1), baseline_pred.reshape(y_pred.shape)):.4f}")

    print("\n Residual ACF (lags 1–5):", np.round(residual_acf(y_true, y_pred, nlags=5)[1:], 3))